<h1><center>Assignment 5 - Data Analytics</center></h1>


<h1><center>Topic: Collaborative Filtering</center></h1>


## Team Name: Mean Team


Name - 1 : Midhush Manohar T.K.  
SRN - 1   : 01FB16ECS208   
Name - 2 : Naveen Suresh  
SRN - 2   : 01FB16ECS222  
Name - 3 : Srikumar Subramanian  
SRN - 3   : 01FB16ECS396   


*Virtually everyone has had an online experience where a website makes personalized recommendations in hopes of future sales or ongoing traffic. Amazon tells you “Customers Who Bought This Item Also Bought”, Udemy tells you “Students Who Viewed This Course Also Viewed”. And Netflix awarded a 1 million dollar prize to a developer team in 2009, for an algorithm that increased the accuracy of the company’s recommendation system by 10 percent.
Recommendation Systems have become such a common part of our everyday lives, and there are a number of methods and techniques involved in building them, Collaborative Filtering being one of the main ones.*


## Question - 1

In [2]:
import pandas as pd
import numpy as np

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [3]:
# a) Average Rating given by each user

train_avg = train_data.groupby('user_id').aggregate({'rating':'mean'})
test_avg = test_data.groupby('user_id').aggregate({'rating':'mean'})

print('Average Rating given by each User')
print('')
print(train_avg.head())
print('')
print('Count (Training Dataset): ' + str(len(train_avg)))
print('\n')
print(test_avg.head())
print('')
print('Count (Testing Dataset): ' + str(len(test_avg)))

Average Rating given by each User

         rating
user_id        
243         7.0
244         7.0
254         7.0
805         8.0
1075        8.0

Count (Training Dataset): 2517


         rating
user_id        
254        10.0
805         8.0
1075        5.0
1261        7.0
1424        8.0

Count (Testing Dataset): 848


In [4]:
# b) Average Rating given for each item

train_item_avg = train_data.groupby('book').aggregate({'rating':'mean'})
test_item_avg = test_data.groupby('book').aggregate({'rating':'mean'})

print("Bookwise Average of Rating on Training Data")
print(train_item_avg.head())
print('')
print(len(train_item_avg))
print('\n')
print("Bookwise Average of Rating on Testing Data")
print(test_item_avg.head())
print('')
print(len(test_item_avg))

Bookwise Average of Rating on Training Data
                                                      rating
book                                                        
1st to Die: A Novel                                 7.607843
A Heartbreaking Work of Staggering Genius           7.488889
A Is for Alibi (Kinsey Millhone Mysteries (Pape...  7.560000
A Lesson Before Dying (Vintage Contemporaries (...  7.411765
A Man Named Dave: A Story of Triumph and Forgiv...  7.885714

100


Bookwise Average of Rating on Testing Data
                                                      rating
book                                                        
1st to Die: A Novel                                 7.000000
A Heartbreaking Work of Staggering Genius           6.444444
A Is for Alibi (Kinsey Millhone Mysteries (Pape...  7.562500
A Lesson Before Dying (Vintage Contemporaries (...  8.000000
A Man Named Dave: A Story of Triumph and Forgiv...  5.666667

100


In [5]:
# c) Function to compute the Root Mean Squared Error

def rmse(y, y_pred):
    mse = np.mean((y - y_pred)**2)
    rmse = np.sqrt(mse)
    return rmse

# RMSE calculation on assigning overall average rating to each user/item pair

# pred = train_data.groupby(['user_id', 'book'])['rating'].mean()
train_data_new = train_data.copy(deep = True)
train_data_new['rating_pred'] = (train_data_new.groupby(['user_id', 'book']).transform(lambda x: train_data['rating'].mean()))
# Each user_id/book pair is unique.
rmse_rating = rmse(train_data_new['rating'], train_data_new['rating_pred'])
print('RMSE on assigning overall average rating to each user_id/book pair is ' + str(rmse_rating))

RMSE on assigning overall average rating to each user_id/book pair is 1.7813364070210418


In [8]:
# d) Convert the train and the test data into a matrix. These matrices, in this format, will be used as input to the 
# functions defined for further predictions. 

train_rows = train_data.user_id.unique()
train_cols = train_data['book'].unique()
train_matrix_data = train_data[['user_id', 'book', 'rating']]

train_idict = dict(zip(train_cols, range(len(train_cols))))
train_udict = dict(zip(train_rows, range(len(train_rows))))

train_matrix_data.user_id = [train_udict[i] for i in train_matrix_data.user_id]
train_matrix_data['book'] = [train_idict[i] for i in train_matrix_data['book']]

train_matrix = train_matrix_data.as_matrix()

# print(train_matrix)


test_rows = test_data.user_id.unique()
test_cols = test_data['book'].unique()
test_matrix_data = test_data[['user_id', 'book', 'rating']]

test_idict = dict(zip(test_cols, range(len(test_cols))))
test_udict = dict(zip(test_rows, range(len(test_rows))))

test_matrix_data.user_id = [test_udict[i] for i in test_matrix_data.user_id]
test_matrix_data['book'] = [test_idict[i] for i in test_matrix_data['book']]

test_matrix = test_matrix_data.as_matrix()

# print(test_matrix)

# Calculate the Sparsity of these matrices.

train_sparsity = 1.0 - np.count_nonzero(train_matrix)/train_matrix.size
print('Sparsity (Train Matrix): ' + str(train_sparsity*100) + ' %')

test_sparsity = 1.0 - np.count_nonzero(test_matrix)/test_matrix.size
print('Sparsity (Test Matrix): ' + str(test_sparsity*100) + ' %')


Sparsity (Train Matrix): 1.5778357559945388 %
Sparsity (Test Matrix): 0.6546644844517169 %


C:\Users\mimos\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\Users\mimos\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [7]:
# e) Write a function to predict in the following manner:
# For any (user i, item j), assign it with: 
#      (mean rating given by user i) + (mean rating received by item j) – (average rating over the entire dataset)
#      i.e., umean[i] + imean[j] – amean

def predict_naive(user, item):
    prediction = item_mean[item] + user_mean[user] - avg_mean
    return prediction

naive = np.zeros((len(train_rows), len(train_cols)))
for row in train_matrix:
    naive[row[0], row[1]] = row[2]

avg_mean = np.mean(naive[naive != 0])
user_mean = sum(naive.T)/sum((naive != 0).T)
item_mean = sum(naive)/sum(naive != 0)

predictions = []
targets = []

for row in train_matrix:
    user, item, actual = row[0], row[1], row[2]
    predictions.append(predict_naive(user, item))
    targets.append(actual)

predictions = np.array(predictions)
targets = np.array(targets)
rmse_value_train = rmse(targets, predictions)

print('RMSE (Train Dataset): ' + str(rmse_value_train))

predictions = []
targets = []

for row in test_matrix:
    user, item, actual = row[0], row[1], row[2]
    predictions.append(predict_naive(user, item))
    targets.append(actual)

predictions = np.array(predictions)
targets = np.array(targets)
rmse_value_test = rmse(targets, predictions)

print('RMSE (Test Dataset): ' + str(rmse_value_test))

RMSE (Train Dataset): 1.1444196071250665
RMSE (Test Dataset): 2.2847566119422518


## Question - 2

In [2]:
import csv
import pandas as pd
import numpy as np
from scipy import stats
import sys
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import *
from scipy.spatial.distance import cosine

data = pd.read_csv('test.csv')
# data.rename(columns={'user_id':'book','book':'user_id', 'rating':'rating'}, inplace = True)

In [3]:
# a) For a user x:
# Find k similar users using cosine similarity and Pearson correlation. Use k = 5

rows = data.user_id.unique()
cols = data['book'].unique()
data = data[['user_id', 'book', 'rating']]
idict = dict(zip(cols, range(len(cols))))
udict = dict(zip(rows, range(len(rows))))
data.user_id = [ udict[i] for i in data.user_id ]
data['book'] = [ idict[i] for i in data['book'] ]
nmat = data.as_matrix()
naive = np.zeros((len(rows),len(cols)))
for row in nmat:
    naive[row[0], row[1]] = row[2]
amean = np.mean(naive[naive != 0])
umean = sum(naive.T) / sum((naive != 0).T)
imean = sum(naive) / sum((naive != 0))

def cos(mat, a, b):
    if a == b:
        return 1
    aval = mat.T[a].nonzero()
    bval = mat.T[b].nonzero()
    corated = np.intersect1d(aval, bval)
    if len(corated) == 0:
        return 0
    avec = np.take(mat.T[a], corated)
    bvec = np.take(mat.T[b], corated)
    val = 1 - cosine(avec, bvec)
    if np.isnan(val):
        return 0
    return val

def pr(mat, a, b, imean):
    if a == b:
        return 1
    aval = mat.T[a].nonzero()
    bval = mat.T[b].nonzero()
    corated = np.intersect1d(aval, bval)
    if len(corated) < 2:
        return 0
    avec = np.take(mat.T[a], corated)
    bvec = np.take(mat.T[b], corated)
    avec1 = avec - imean[a]
    bvec1 = bvec - imean[b]
    val = 1 - cosine(avec1, bvec1)
    if np.isnan(val):
        return 0
    return val

def itemsimilar(mat, option=""):
    amean = np.mean(mat[mat!=0])
    umean = sum(mat.T) / sum((mat!=0).T)
    imean = sum(mat) / sum((mat!=0))
    n = mat.shape[1]
    sim_mat=np.zeros((n,n))
    if option == 'pr':
        for i in range(n):
            for j in range(n):
                sim_mat[i][j] = pr(mat, i, j, imean)
        sim_mat = (sim_mat + 1)/2
    elif option == 'cos':
        for i in range(n):
            for j in range(n):
                sim_mat[i][j] = cos(mat, i, j)
    else:
        sim_mat = cosine_similarity(mat.T)
    return sim_mat, amean, umean, imean

sim_mat, amean, umean, imean = itemsimilar(naive,"")
print('Similarity Matrix: ')
print(sim_mat)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 1.]]


C:\Users\mimos\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()


In [6]:
#b) Write a function to predict the rating for the user x and item j, based on K similar user to x.

def predict(user, item, mat, item_similarity, amean, umean, imean, k=20):
    nzero = mat[user].nonzero()[0]
    if len(nzero) == 0:
        return amean
    baseline = imean + umean[user] - amean
    choice = nzero[item_similarity[item, nzero].argsort()[::-1][:k]]
    prediction = ((mat[user, choice] - baseline[choice]).dot(item_similarity[item, choice])/sum(item_similarity[item, choice]))+ baseline[item]
    if np.isnan(prediction):
        prediction = amean
    if prediction > 10:
        prediction = 10
    if prediction < 1:
        prediction = 1
    return prediction

prediction_value = predict(1, 1, naive, sim_mat, amean, umean, imean, k=5)
print('Predicted Value: ' + str(prediction_value))
print('Naive Matrix: ')
print(naive)

Predicted Value: 8.000000000000002
Naive Matrix: 
[[7. 0. 0. ... 0. 0. 0.]
 [0. 8. 0. ... 0. 0. 0.]
 [0. 0. 8. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [7]:
#c) Report the train and test RMSE errors for these predictions

def get_results(train_data, test_data, option, k):
    # train_data.rename(columns={'user_id':'book','book':'user_id', 'rating':'rating'}, inplace = True)
    rows = train_data.user_id.unique()
    cols = train_data['book'].unique()
    train_data = train_data[['user_id', 'book', 'rating']]
    idict = dict(zip(cols, range(len(cols))))
    udict = dict(zip(rows, range(len(rows))))
    train_data.user_id = [ udict[i] for i in train_data.user_id ]
    train_data['book'] = [ idict[i] for i in train_data['book'] ]
    mat = train_data.as_matrix()
    full_mat = np.zeros((len(rows),len(cols)))
    for row in mat:
        full_mat[row[0], row[1]] = row[2]
    item_similarity, amean, umean, imean = itemsimilar(full_mat, option)
    preds=[]
    real=[]
    for i in mat:
        preds.append(predict(i[0], i[1], full_mat, item_similarity, amean, umean, imean, k=5))
        real.append(i[2])
    err1 = mean_squared_error(preds,real)
    print('Train Error')
    print('RMSE : %.4f' % err1)

    # test_data.rename(columns={'user_id':'book','book':'user_id', 'rating':'rating'}, inplace = True)
    rows = test_data.user_id.unique()
    cols = test_data['book'].unique()
    test_data = test_data[['user_id', 'book', 'rating']]
    idict = dict(zip(cols, range(len(cols))))
    udict = dict(zip(rows, range(len(rows))))
    test_data.user_id = [ udict[i] for i in test_data.user_id ]
    test_data['book'] = [ idict[i] for i in test_data['book'] ]
    mat = test_data.as_matrix()
    full_mat = np.zeros((len(rows),len(cols)))
    for row in mat:
        full_mat[row[0], row[1]] = row[2]
    item_similarity, amean, umean, imean = itemsimilar(full_mat, option)
    preds=[]
    real=[]
    for i in mat:
        preds.append(predict(i[0], i[1], full_mat, item_similarity, amean, umean, imean, k=5))
        real.append(i[2])
    err2 = mean_squared_error(preds,real)
    print('Test Error')
    print('RMSE : %.4f' % err2)
    
get_results(pd.read_csv("train.csv"),pd.read_csv("test.csv"), "cos", 5)

C:\Users\mimos\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


Train Error
RMSE : 0.9937


C:\Users\mimos\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


Test Error
RMSE : 0.5875


## Question 2d

Firstly, the value of K is highly data dependent. In general, a larger k suppresses the effects of noise, but makes the classification boundaries less distinct.

There are two approaches that are usually followed when it comes to choosing K values:
1. Try many K values and use k-fold Cross-Validation to see which K value gives the best accuracy. The goal of cross-validation is to test the model’s ability to predict new data that was not used in estimating it, in order to flag problems like overfitting and to give an insight on how the model will generalize to an independent dataset. 
2. Alternatively, one can use the Elbow method. This method looks at the percentage of variance explained as a function of the number of clusters. One should choose a number of clusters so that adding another cluster doesn't give much better modeling of the data.
